In [47]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [48]:
import torch
import sklearn

In [49]:
# seed 고정
torch.manual_seed(1)

In [50]:
# data load
train = pd.read_csv("/kaggle/input/2021-ai-quiz1-p4/train.csv")
test = pd.read_csv("/kaggle/input/2021-ai-quiz1-p4/test.csv")
submit = pd.read_csv("/kaggle/input/2021-ai-quiz1-p4/submit_sample.csv")

In [51]:
# 데이터 확인

print(train)
print(test)
print(submit)

In [52]:
## data 전처리 및 텐서 올리기
# 스케일 조정을 위한 Scaler

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

x_train = np.array(train.drop(['id','count'],axis=1))
x_test = np.array(test.drop(['id'],axis=1))
y_train = np.array(train['count'])

# scale 조정
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

x_train = torch.Tensor(x_train).cuda()
x_test = torch.Tensor(x_test).cuda()
y_train = torch.Tensor(y_train).unsqueeze(1).cuda()

print(x_train.shape,y_train.shape)

In [53]:
# 학습 설정
w = torch.zeros((9,1),requires_grad=True,device="cuda")
b = torch.zeros(1,requires_grad=True,device="cuda")

optim = torch.optim.SGD([w,b],lr = 1e-3)
epochs= 4000
# 회귀 문제 이므로 MSE 사용
loss = torch.nn.MSELoss()

In [54]:
for epoch in range(epochs+1):
    optim.zero_grad()
    
    h = x_train.matmul(w)+b
    cost = loss(h,y_train)
    
    cost.backward()
    optim.step()
    
    if epoch % 1000 == 0:
        print(cost.item())

In [55]:
with torch.no_grad():
    predict = x_test.matmul(w) +b
predict = predict.cpu().numpy()
submit['count'] = predict
submit.to_csv("submission.csv",index=False)

# try
# lr 1e-4 일때는 코스트 감소가 너무 저조함
# lr 1e-3 일때 코스트 감소가 잘되지만 언제 고정되는 지 알 수 없음
# epoch를 10000으로 증가해보니 3~4000쯤 2921로 코스트 고정됨 -> epoch 4000 으로 사용